In [83]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [84]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [85]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [86]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

In [87]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
@variable(model, delta_up[1:W,1:hours])
@variable(model, delta_down[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W*(
        scenarios[i][hour,"price"] * p_DA[hour]
     + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
      - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
      ) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)



# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

In [88]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))
println("delta_up: ", value.(delta_up))
println("delta_down: ", value.(delta_down))

In [89]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])
#, delta = Float64[], delta_up = Float64[], delta_down = Float64[]

#make a table with the results
scenario = 1
for hour in 1:hours
    push!(result_df, [hour, value(p_DA[hour])])
    #, value(delta[scenario,hour]), value(delta_up[scenario,hour]), value(delta_down[scenario,hour])
end

pretty_table(result_df, backend = Val(:latex))

In [90]:
profit_df = DataFrame(scenario = Int[], profit = Float64[])

for i in 1:W
    profit = 0
    for hour in 1:hours
        profit += (
            scenarios[i][hour,"price"] * value(p_DA[hour])
        + value(delta_up[i,hour]) * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
        - value(delta_down[i,hour]) * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
        )
    end
    push!(profit_df, [i, profit])
end

#return average of profit
println("Average profit: ", sum(profit_df[!,"profit"])/W)

In [91]:
CSV.write("1_2_results/profits_1_2.csv", profit_df)
CSV.write("1_2_results/market_results_1_2.csv", result_df)

"1_2_results/market_results_1_2.csv"